In [1]:
### 사전준비
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from matplotlib import font_manager, rc
import platform
import json
import folium
if platform.system() == "Windows" :
    path = "c:/Windows/Fonts/Malgun.ttf"
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc("font", family = font_name)
elif platform.system() == "Drawin" :
    rc("font",family = "AppleGothic")
else :
    print("check your OS system...")
plt.rcParams["axes.unicode_minus"] = False
from bs4 import BeautifulSoup
import requests
from openpyxl.workbook import Workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import random

In [2]:
df = pd.read_excel("./데이터_변수정리_찐최종본.xlsx",
                   header =1, usecols=[1,6,14], names = ['val1', 'val2', 'val3'])

# 17년도 데이터 프레임 불러오기
val_lis = list(df['val2'].dropna(axis = 0))
val_lis2 = []

for val in val_lis:
    if 'yq9a' in val:
        temp = ['yq9a{}'.format(str(i).zfill(2)) for i in range(1, 18)]
        temp = temp+['yq9a97', 'yq9a99']
        
    elif '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[0-9]{0,3}$').search(val).start()
        i_end = re.compile('[0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1,int(ran)+1):
            val_lis2.append('{}{}'.format(val[:text],i))
            
    else:
        val_lis2.append(val)

val_lis2 = val_lis2+temp

df17 = pd.read_csv("../01_raw데이터/2017년_원자료.csv").loc[:,val_lis2]

# 18년도 데이터 프레임 불러오기
val_lis = list(df['val3'].dropna(axis = 0))
val_lis2 = []

for val in val_lis:
    if '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start()+1
        i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1,int(ran)+1):
            val_lis2.append('{}{}'.format(val[:text],i))
            
    else:
        val_lis2.append(val)

df18 = pd.read_csv("../01_raw데이터/2018년_원자료.csv").loc[:,val_lis2]

# 19년도 데이터 프레임 불러오기
val_lis = list(df['val1'].dropna(axis = 0))
val_lis2 = []

for val in val_lis:
    if '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start()+1
        i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1,int(ran)+1):
            val_lis2.append('{}{}'.format(val[:text],i))
            
    else:
        val_lis2.append(val)

df19 = pd.read_csv("../01_raw데이터/2019년_원자료.csv").loc[:,val_lis2]

In [3]:
# 최초방문인 관광객들 1회방문 넣어주기
df17["wq1a"].fillna(1,inplace=True)

In [4]:
# 응답항목 기준에 맞게 바꿔주기
df17[["q5a1","q5a2","q5a3"]] = df17[["q5a1","q5a2","q5a3"]].replace({1:3,2:2,3:4,4:1,5:7,6:7,7:5,8:3,9:12,10:11,11:13,12:13,13:6,14:8,15:8,16:10,17:13})

In [5]:
# 1~10까지 나눠줄 4번을 빼고 바꿀 값으로 바꿔주기
df17[["q8a1","q8a2","q8a3"]] = df17[["q8a1","q8a2","q8a3"]].replace(4,"n")
df17[["q8a1","q8a2","q8a3"]] = df17[["q8a1","q8a2","q8a3"]].replace({1:2,2:1,3:9,5:9,6:9,7:8,8:8,9:8,10:10,11:10,12:11,13:13,14:12,15:11,16:11,17:4,18:3,19:5,20:6,21:14,22:5,23:14})

In [6]:
# 랜덤할당
import random

def ratelist(row_name, val_rmd):
    li = []
    
    if row_name == 'Q9_2A1':
        dic = {1.0:0.318198, 2.0:0.214775, 3.0:0.134685, 10.0:0.101622,
                4.0:0.092523, 6.0:0.064324, 5.0:0.034054,7.0:0.017477,
                8.0:0.013423, 9.0:0.008919}
    elif row_name == 'Q9_2A2':
        dic = {2.0:0.328732,1.0:0.192887,3.0:0.145144,4.0:0.110057,
                10.0:0.072189,5.0:0.066053,7.0:0.031636,6.0:0.030678,
                8.0:0.011888,9.0:0.010737}
    else:
        dic = {3.0:0.176084,2.0:0.157057,1.0:0.156711,4.0:0.129958,
                10.0:0.127076,5.0:0.111624,6.0:0.054659,7.0:0.047970,
                8.0:0.024331,9.0:0.014530}
    
    rate = list(dic.values())
    total = len(df17[df17[row_name]=="n"])
    count=0

    for i in range(1,11):
        num = int(round(dic[i],len(str(total)))*total)
        count+=num
        for j in range(0, num):
            li.append(i)
            
    if count != total:
        for k in range(0,total-count):
            li.append(int(list(dic.keys())[0]))
            
    random.shuffle(li)
    
    for l, m in zip(range(0,len(li)), df17[df17[row_name]==val_rmd].index):
        df17.loc[m,row_name] = li[l]

In [7]:
# 4번("n") 비율별로 나눠주기
ratelist("q8a1","n")
ratelist("q8a2","n")
ratelist("q8a3","n")

In [8]:
df17["q8a3"].value_counts()

q8a3
2     2218
1     1737
8     1522
3     1481
4     1398
5      769
6      675
9      551
10     352
14     320
11     242
12     136
13      91
7       26
Name: count, dtype: int64

In [9]:
col = ['yq9a{}'.format(str(i).zfill(2)) for i in range(1,18)]
(df17.loc[df17["yq9a97"]==97,col].isnull().sum(axis=1) ==17).sum()

0

In [10]:
# 방문지역(17개시도) 무응답 및 기타 삭제(사유: 없거나 미미한 수치)
del df17["yq9a97"]
del df17["yq9a99"]

In [11]:
# 결측치가 있는 항목(숙박비,거주국 지불경비) 0으로 채움
df17[["mtzq19a1","mtzq19a2"]] = df17[["mtzq19a1","mtzq19a2"]].fillna(0)

In [12]:
# 19년도처럼 문화+오락 비용 합치기
df17["mtzq19a6+8"] = df17["mtzq19a6"]+df17["mtzq19a8"]

In [13]:
# 금액 수치를 19년도의 범주에 맞춰 변경
for c in ["mtzq19a1","mtzq19a2"] :
    for i in range(len(df17)) :
        if df17.loc[i,c] <= 100 :
            df17.loc[i,c] = 1
        elif df17.loc[i,c] > 100 and df17.loc[i,c] <= 200 :
            df17.loc[i,c] = 2
        elif df17.loc[i,c] > 200 and df17.loc[i,c] <= 500 :
            df17.loc[i,c] = 3
        elif df17.loc[i,c] > 500 and df17.loc[i,c] <= 1000 :
            df17.loc[i,c] = 4
        elif df17.loc[i,c] > 1000 and df17.loc[i,c] <= 10000 :
            df17.loc[i,c] = 5
        elif df17.loc[i,c] > 10000 :
            df17.loc[i,c] = 98

In [14]:
for c in ["mtzq19a3","mtzq19a5"] :
    for i in range(len(df17)) :
        if df17.loc[i,c] <= 50 :
            df17.loc[i,c] = 1
        elif df17.loc[i,c] > 50 and df17.loc[i,c] <= 100 :
            df17.loc[i,c] = 2
        elif df17.loc[i,c] > 100 and df17.loc[i,c] <= 200 :
            df17.loc[i,c] = 3
        elif df17.loc[i,c] > 200 and df17.loc[i,c] <= 500 :
            df17.loc[i,c] = 4
        elif df17.loc[i,c] > 500 and df17.loc[i,c] <= 10000 :
            df17.loc[i,c] = 5
        elif df17.loc[i,c] > 10000 :
            df17.loc[i,c] = 98

In [15]:
for c in ["mtzq19a4","mtzq19a7","mtzq19a10","mtzq19a6+8"] :
    for i in range(len(df17)) :
        if df17.loc[i,c] == 0 :
            df17.loc[i,c] = 1
        elif df17.loc[i,c] > 0 and df17.loc[i,c] <= 30 :
            df17.loc[i,c] = 2
        elif df17.loc[i,c] > 30 and df17.loc[i,c] <= 100 :
            df17.loc[i,c] = 3
        elif df17.loc[i,c] > 100 and df17.loc[i,c] <= 10000 :
            df17.loc[i,c] = 4
        elif df17.loc[i,c] > 10000 :
            df17.loc[i,c] = 5

In [16]:
# 방문목적을 19년도의 범주에 맞게 변경
df17[["q5"]] = df17[["q5"]].replace({1:1,2:6,3:5,4:1,5:2,6:3,7:4,8:6})

In [17]:
# 방문지의 값을 한글로 변경
file_path = "./데이터_변수정리_찐최종본.xlsx"
df17_val = pd.read_excel(file_path,\
                           usecols = 'A : B', sheet_name = '17년여행지변수명')
df18_val = pd.read_excel(file_path,\
                           usecols = 'A : B', sheet_name = '18년여행지변수명')
df19_val = pd.read_excel(file_path,\
                           usecols = 'A : B', sheet_name = '19년여행지변수명')

for i in range(813) : # 17년여행지변수명의 row 갯수가 813개
    df17 = df17.replace({'q11a1' : df17_val.loc[i, '변수명']}, df17_val.loc[i, '변수내용'])
    df17 = df17.replace({'q11a2' : df17_val.loc[i, '변수명']}, df17_val.loc[i, '변수내용'])
    df17 = df17.replace({'q11a3' : df17_val.loc[i, '변수명']}, df17_val.loc[i, '변수내용'])
    
for j in range(357) : # 18년여행지변수명의 row 갯수가 357개
    df18 = df18.replace({'Q11A1' : df18_val.loc[j, '변수명']}, df18_val.loc[j, '변수내용'])
    df18 = df18.replace({'Q11A2' : df18_val.loc[j, '변수명']}, df18_val.loc[j, '변수내용'])
    df18 = df18.replace({'Q11A3' : df18_val.loc[j, '변수명']}, df18_val.loc[j, '변수내용'])
    
for k in range(365) : # 19년여행지변수명의 row 갯수가 365개
    df19 = df19.replace({'Q10_2A1' : df19_val.loc[k, '변수명']}, df19_val.loc[k, '변수내용'])
    df19 = df19.replace({'Q10_2A2' : df19_val.loc[k, '변수명']}, df19_val.loc[k, '변수내용'])
    df19 = df19.replace({'Q10_2A3' : df19_val.loc[k, '변수명']}, df19_val.loc[k, '변수내용'])

In [18]:
# 연령대가 9번(기타)인 값들 제거
df_2017 = df17[df17["age"] != 9].reset_index(drop = True)

In [19]:
# 만족도의 9번(해당 없음)을 19년도처럼 0번으로 교체
for i in range(10) :
    df_2017 = df_2017.replace({f'q13a{str(i + 1).zfill(2)}' : 9.0}, 0)

In [20]:
# 문화, 오락 비용 개별 컬럼 제거, 문화+오락 컬럼 사용
df_2017 = df_2017.drop(['mtzq19a6','mtzq19a8'], axis=1)
df_2017.rename(columns={'mtzq19a6+8':'mtzq19a6'},inplace=True)

In [21]:
# 19년도와 컬럼명 동기화
file_path = "./데이터_변수정리_찐최종본.xlsx"
df17_val = pd.read_excel(file_path,\
usecols = 'A : B', sheet_name = '변수명매칭')

for i in range(58) :
    df_2017 = df_2017.rename(columns={df17_val.loc[i, '17변수명']: df17_val.loc[i, '19변수명']})
df_2017

,Q1,Q2A1,Q2A2,Q2A3,Q2A4,Q2A5,Q5_2A1,Q5_2A2,Q5_2A3,Q9_2A1,...,Q10_1B9,Q10_1B10,Q10_1B11,Q10_1B12,Q10_1B13,Q10_1B14,Q10_1B15,Q10_1B16,Q10_1B17,CQ14_2_7
0,1.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,18.0,NaN,2.0,NaN,NaN,NaN,3.0,4.0,2.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1.0
2,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,...,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0
3,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0
4,1.0,1.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13819,1.0,NaN,NaN,3.0,NaN,NaN,12.0,3.0,13.0,1,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
13820,1.0,NaN,NaN,3.0,NaN,NaN,3.0,2.0,4.0,1,...,NaN,15.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
13821,3.0,NaN,NaN,3.0,NaN,NaN,7.0,3.0,2.0,3,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
13822,1.0,NaN,NaN,3.0,NaN,NaN,4.0,2.0,3.0,1,...,NaN,15.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0


In [22]:
# 월이 들어간 컬럼을 사용해 년-월 컬럼 생성
for i in range(len(df_2017)) :
    df_2017.loc[i,"D_YEAR"] = "2017-{}".format(str(int(df_2017["D_MON"][i])).zfill(2))
df_2017["D_YEAR"]

0        2017-03
1        2017-11
2        2017-06
3        2017-02
4        2017-08
          ...   
13819    2017-12
13820    2017-12
13821    2017-12
13822    2017-12
13823    2017-12
Name: D_YEAR, Length: 13824, dtype: object

In [23]:
# 방문 지역부분 변수명을 19년도에 맞춰 변경
dic1={}
w_list = [1,4,8,9,6,10,11,17,14,15,3,7,2,5,12,13,16]
for i in range(1,18) :
    dic1["Q10_1B{}".format(i)]="Q10_1B{}".format(w_list[i-1])

df_2017.rename(columns = dic1,inplace=True)

In [24]:
df_2017["D_AGE"].unique()

array([0., 3., 1., 2., 4., 5.])

In [27]:
df_2017['D_AGE'].value_counts()

D_AGE
1.0    5473
2.0    3511
3.0    2212
4.0    1253
0.0     933
5.0     442
Name: count, dtype: int64

In [28]:
# 나이의 범주(0~5)를 19년도처럼 변경(1~6)
df_2017["D_AGE"] = df_2017["D_AGE"]+1

In [29]:
df_2017['D_AGE'].value_counts()

D_AGE
2.0    5473
3.0    3511
4.0    2212
5.0    1253
1.0     933
6.0     442
Name: count, dtype: int64

In [30]:
# 수정 후 저장
df_2017.to_csv("./2017년_변수정리.csv",index=False)